In [1]:
import numpy as np
import pandas as pd 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
os.path.join(PROJECT_ROOT_DIR)
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
CLEAREDDATA_PATH = os.path.join(PROJECT_ROOT_DIR, "cleareddata")
os.makedirs(IMAGES_PATH, exist_ok=True)
os.makedirs(CLEAREDDATA_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

from pipe_classes import *

./data/scotch_review.csv
./data/nyc_benchmarking_disclosure_2017_consumption_data.xlsx


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
data = pd.read_excel(PROJECT_ROOT_DIR+'/data/nyc_benchmarking_disclosure_2017_consumption_data.xlsx','Information and Metrics')

In [3]:
data = data.replace({'Not Available': np.nan})

In [4]:
data.select_dtypes(exclude=['number']).keys()

Index(['Property Name', 'Parent Property Id', 'Parent Property Name',
       'NYC Borough, Block and Lot (BBL) self-reported',
       'NYC Building Identification Number (BIN)', 'Address 1 (self-reported)',
       'Address 2 (self-reported)', 'Postal Code', 'Street Number',
       'Street Name', 'Borough', 'DOF Gross Floor Area (ft²)',
       'Primary Property Type - Self Selected',
       'List of All Property Use Types at Property',
       'Largest Property Use Type', '2nd Largest Property Use Type',
       '3rd Largest Property Use Type', 'Metered Areas (Energy)',
       'Metered Areas  (Water)', 'Annual Maximum Demand (MM/YYYY)',
       'Water Required?', 'Generation Date',
       'DOF Benchmarking Submission Status'],
      dtype='object')

In [5]:
data.select_dtypes('number').keys().size

37

### First exercise: SGD without PCA or collinear features remover and without the sqrt-log feature adder

In [9]:
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

reg_target="ENERGY STAR Score"
spec_cols=['Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_Encoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(0, 96)
(24724, 96)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
X = num_pipeline.fit_transform(X)
X_test = num_pipeline.fit_transform(X_test)
X.shape

(17306, 95)

In [11]:
def results_display(reg, X, X_test, y, y_test):
    from sklearn.metrics import mean_squared_error, r2_score
    from timeit import default_timer as timer
    start = timer()
    reg.fit(X, y)
    #train set
    X_pred = reg.predict(X)
    X_rmse = np.sqrt(mean_squared_error(y, X_pred))
    X_R2 = r2_score(y, X_pred)
    #test set
    X_test_pred = reg.predict(X_test)
    X_test_rmse = np.sqrt(mean_squared_error(y_test, X_test_pred))
    X_test_R2 = r2_score(y_test, X_test_pred)
    end = timer()
    print(str(reg))
    print('Train set: Regressor name, RMSE, R2')
    print(reg.__class__.__name__, round(X_rmse,3), round(X_R2,3))
    print('Test set: Regressor name, RMSE, R2')
    print(reg.__class__.__name__, round(X_test_rmse,3), round(X_test_R2,3))
    print('Time consumption [s]: ', round((end - start),3))
    print('---------------------------------------------------------')

In [12]:
from sklearn.linear_model import SGDRegressor

sgd_default = SGDRegressor(loss='squared_loss', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, 
               tol=0.001, shuffle=True, verbose=0, epsilon=0.1, random_state=4711, learning_rate='invscaling', 
               eta0=0.01, power_t=0.25, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, 
               warm_start=False, average=False)

sgd_loss_huber = SGDRegressor(loss='huber', random_state=4711)
sgd_loss_eps = SGDRegressor(loss='epsilon_insensitive',random_state=4711) #gnores errors less than epsilon and is linear past that; this is the loss function used in SVR
sgd_loss_sqeps = SGDRegressor(loss='squared_epsilon_insensitive',random_state=4711)

for reg in (sgd_default, sgd_loss_huber, sgd_loss_eps, sgd_loss_sqeps):
    results_display(reg, X, X_test, y, y_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=4711,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)
Train set: Regressor name, RMSE, R2
SGDRegressor 799794402.506 -625244197081064.1
Test set: Regressor name, RMSE, R2
SGDRegressor 752049612.022 -557196386139178.56
Time consumption [s]:  0.149
---------------------------------------------------------
SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='huber', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=4711,
             shuffle=True, tol=0.001, validation_fraction=0.1, ve

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=4711, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Train set: Regressor name, RMSE, R2
SGDRegressor 17.586 0.698
Test set: Regressor name, RMSE, R2
SGDRegressor 17.952 0.683
Time consumption [s]:  2.079
---------------------------------------------------------
SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=4711, shuffle=True, tol=0.001,
             validation_fraction=0.1, v

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
from sklearn.model_selection import GridSearchCV
from timeit import default_timer as timer

start = timer()
param_grid = [
    # try 10 (2×5) combinations of hyperparameters
    {'loss': ['huber', 'epsilon_insensitive'], 'epsilon': [0.0001, 0.0003, 0.001, 0.003, 0.01]},
    ]

sgd_reg = SGDRegressor()
# train across 5 folds, that's a total of 10*5=50 rounds of training 
grid_search = GridSearchCV(sgd_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
end = timer()
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')
print('Full time consumption [s]: ', round((end - start),3))
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best estimator: SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.0001,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Best params: {'epsilon': 0.0001, 'loss': 'epsilon_insensitive'}
---------------------------------------------------------------------
63.85040190807861 {'epsilon': 0.0001, 'loss': 'huber'}
17.6645774215665 {'epsilon': 0.0001, 'loss': 'epsilon_insensitive'}
63.83914875589347 {'epsilon': 0.0003, 'loss': 'huber'}
17.666783212944026 {'epsilon': 0.0003, 'loss': 'epsilon_insensitive'}
63.79985267236803 {'epsilon': 0.001, 'loss': 'huber'}
17.671406212610997 {'epsilon': 0.001, 'loss': 'epsilon_insensitive'}
63.6873226634118 {'epsilon': 0.003, 'loss': 'huber'}
17.667386147755444

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.0001,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Train set: Regressor name, RMSE, R2
SGDRegressor 17.578 0.698
Test set: Regressor name, RMSE, R2
SGDRegressor 17.953 0.682
Time consumption [s]:  2.339
---------------------------------------------------------
---------------------------------------------------------------------
Full time consumption [s]:  52.64
---------------------------------------------------------------------


In [17]:
start = timer()
param_grid = [
    # try 16 (4×4) combinations of hyperparameters
    {'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'], 'eta0': [0.005, 0.01, 0.02, 0.04]},
    ]

sgd_reg = SGDRegressor(loss='epsilon_insensitive', epsilon=0.001)
# train across 5 folds, that's a total of 16*5=80 rounds of training 
grid_search = GridSearchCV(sgd_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
end = timer()
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')
print('Full time consumption [s]: ', round((end - start),3))
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best estimator: SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.001,
             eta0=0.04, fit_intercept=True, l1_ratio=0.15,
             learning_rate='adaptive', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Best params: {'eta0': 0.04, 'learning_rate': 'adaptive'}
---------------------------------------------------------------------
17.683593349215176 {'eta0': 0.005, 'learning_rate': 'constant'}
17.781842106330764 {'eta0': 0.005, 'learning_rate': 'optimal'}
17.72145981142665 {'eta0': 0.005, 'learning_rate': 'invscaling'}
17.600690876980906 {'eta0': 0.005, 'learning_rate': 'adaptive'}
17.732304556219226 {'eta0': 0.01, 'learning_rate': 'constant'}
17.897652948957226 {'eta0': 0.01, 'learning_rate': 'optimal'}
17.675769082907834 {'eta0': 0.01, 'learning_rate': 'invscaling'}
17.596

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.001,
             eta0=0.04, fit_intercept=True, l1_ratio=0.15,
             learning_rate='adaptive', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Train set: Regressor name, RMSE, R2
SGDRegressor 17.533 0.7
Test set: Regressor name, RMSE, R2
SGDRegressor 18.0 0.681
Time consumption [s]:  1.078
---------------------------------------------------------
---------------------------------------------------------------------
Full time consumption [s]:  81.268
---------------------------------------------------------------------


In [20]:
from sklearn.ensemble import AdaBoostRegressor

ada_SGD = AdaBoostRegressor(base_estimator=grid_best_est, n_estimators=150, random_state=4711)

results_display(ada_SGD, X, X_test, y, y_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(base_estimator=SGDRegressor(alpha=0.0001, average=False,
                                              early_stopping=False,
                                              epsilon=0.001, eta0=0.04,
                                              fit_intercept=True, l1_ratio=0.15,
                                              learning_rate='adaptive',
                                              loss='epsilon_insensitive',
                                              max_iter=1000, n_iter_no_change=5,
                                              penalty='l2', power_t=0.25,
                                              random_state=None, shuffle=True,
                                              tol=0.001,
                                              validation_fraction=0.1,
                                              verbose=0, warm_start=False),
                  learning_rate=1.0, loss='linear', n_estimators=150,
                  random_state=4711)
Train set: Re

### Now try a PCA instead of the collinear features removal

In [21]:
prep_pipeline2 = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_adderencoder', Feature_AdderEncoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

In [22]:
X, X_test, y, y_test = prep_pipeline2.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/notebooks/ML/pipe_classes.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
/notebooks/ML/pipe_classes.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['log_' + col] = np.log(numeric_subset[col])
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856:

(0, 144)
(24724, 144)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
X = num_pipeline.fit_transform(X)
X_test = num_pipeline.fit_transform(X_test)
X.shape

(17306, 143)

In [24]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_reduced = pca.fit(X)
pca.explained_variance_ratio_

array([0.21151139, 0.10719866, 0.07644481, 0.04385626, 0.03542039,
       0.03017877, 0.0281349 , 0.02749113, 0.02466106, 0.02327311,
       0.02065438, 0.0185173 , 0.01581245, 0.01281392, 0.01214799,
       0.01148795, 0.01123512, 0.01065479, 0.01039217, 0.01027884,
       0.01017831, 0.01016064, 0.01015428, 0.01013967, 0.01011885,
       0.01011348, 0.01010649, 0.0101055 , 0.01010232, 0.01010156,
       0.01009278, 0.01007499, 0.01003872, 0.01001209, 0.00993724,
       0.00986861, 0.00973461, 0.00950228, 0.00882001, 0.00767634,
       0.00748354, 0.00695605])

In [25]:
1 - pca.explained_variance_ratio_.sum()

0.046356253476518394

In [26]:
X_reduced = pca.transform(X)
X_test_reduced = pca.transform(X_test)
X_reduced.shape

(17306, 42)

In [27]:
start = timer()
param_grid = [
    # try 20 (4×5) combinations of hyperparameters
    {'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'], 'eta0': [0.001, 0.005, 0.01, 0.02, 0.04]},
    ]

sgd_reg = SGDRegressor(loss='epsilon_insensitive', epsilon=0.001)
# train across 5 folds, that's a total of 16*5=80 rounds of training 
grid_search = GridSearchCV(sgd_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X_reduced, y)
end = timer()
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')
print('Full time consumption [s]: ', round((end - start),3))
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This wi

Best estimator: SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.001,
             eta0=0.001, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Best params: {'eta0': 0.001, 'learning_rate': 'invscaling'}
---------------------------------------------------------------------
19.559593630258625 {'eta0': 0.001, 'learning_rate': 'constant'}
46.419352974222946 {'eta0': 0.001, 'learning_rate': 'optimal'}
19.245761665032415 {'eta0': 0.001, 'learning_rate': 'invscaling'}
19.60862426068047 {'eta0': 0.001, 'learning_rate': 'adaptive'}
20.09818424561093 {'eta0': 0.005, 'learning_rate': 'constant'}
46.59288461725372 {'eta0': 0.005, 'learning_rate': 'optimal'}
19.288625412839416 {'eta0': 0.005, 'learning_rate': 'invscaling'}

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.001,
             eta0=0.001, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Train set: Regressor name, RMSE, R2
SGDRegressor 16.877 0.722
Test set: Regressor name, RMSE, R2
SGDRegressor 17.356 0.703
Time consumption [s]:  9.631
---------------------------------------------------------
---------------------------------------------------------------------
Full time consumption [s]:  55.149
---------------------------------------------------------------------


to conclude, we see three main points:
- PCA doesn't reduce the dimensions as much as the collinear features remover
- low learning rates are still very helpful
- PCA is as good as the the collinear features remover (best: train set: cfr -> 63.4%; pca -> 64.3%; test set: crf -> 62.3%; pca -> 61.9%)
- Full time consumption on Thinkpad Core i7 is ~55s (first exercise without pca: Full time consumption on Thinkpad Core i7 is ~80s)

### Now just do the same exercise without PCA or collinear features remover

In [28]:
X, X_test, y, y_test = prep_pipeline2.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/notebooks/ML/pipe_classes.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
/notebooks/ML/pipe_classes.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['log_' + col] = np.log(numeric_subset[col])
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856:

(0, 144)
(24724, 144)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
X = num_pipeline.fit_transform(X)
X_test = num_pipeline.fit_transform(X_test)
X.shape

(17306, 143)

In [30]:
start = timer()
param_grid = [
    # try 20 (4×5) combinations of hyperparameters
    {'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'], 'eta0': [0.001, 0.005, 0.01, 0.02, 0.04]},
    ]

sgd_reg = SGDRegressor(loss='epsilon_insensitive', epsilon=0.001)
# train across 5 folds, that's a total of 16*5=80 rounds of training 
grid_search = GridSearchCV(sgd_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
end = timer()
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')
print('Full time consumption [s]: ', round((end - start),3))
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best estimator: SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.001,
             eta0=0.04, fit_intercept=True, l1_ratio=0.15,
             learning_rate='adaptive', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Best params: {'eta0': 0.04, 'learning_rate': 'adaptive'}
---------------------------------------------------------------------
16.78211727861609 {'eta0': 0.001, 'learning_rate': 'constant'}
16.885650244724363 {'eta0': 0.001, 'learning_rate': 'optimal'}
17.03217933821355 {'eta0': 0.001, 'learning_rate': 'invscaling'}
16.748640350576913 {'eta0': 0.001, 'learning_rate': 'adaptive'}
16.736991265421416 {'eta0': 0.005, 'learning_rate': 'constant'}
17.27262409368566 {'eta0': 0.005, 'learning_rate': 'optimal'}
16.87946589095112 {'eta0': 0.005, 'learning_rate': 'invscaling'}
16.742

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.001,
             eta0=0.04, fit_intercept=True, l1_ratio=0.15,
             learning_rate='adaptive', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Train set: Regressor name, RMSE, R2
SGDRegressor 16.57 0.732
Test set: Regressor name, RMSE, R2
SGDRegressor 17.302 0.705
Time consumption [s]:  2.013
---------------------------------------------------------
---------------------------------------------------------------------
Full time consumption [s]:  203.69
---------------------------------------------------------------------


to conclude, we see two main points:
- low learning rates are still very helpful
- PCA and collinear features remover are making the regression worse by quite a factor (best: train set: w.o. -> 73.2%; pca/cfr -> 64%; test set: w.o. -> 71.0%; pca/cfr -> 62%)
- however, full time consumption on Thinkpad Core i7 is ~200s

### Now just do the same exercise without PCA or collinear features remover and without the sqrt-log feature adder

In [31]:
spec_cols=['Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']
prep_pipeline3 = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_Encoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

In [32]:
X, X_test, y, y_test = prep_pipeline3.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(0, 96)
(24724, 96)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
X = num_pipeline.fit_transform(X)
X_test = num_pipeline.fit_transform(X_test)
X.shape

(17306, 95)

In [34]:
start = timer()
param_grid = [
    # try 20 (4×5) combinations of hyperparameters
    {'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'], 'eta0': [0.005, 0.01, 0.02, 0.04, 0.08]},
    ]

sgd_reg = SGDRegressor(loss='epsilon_insensitive', epsilon=0.001)
# train across 5 folds, that's a total of 16*5=80 rounds of training 
grid_search = GridSearchCV(sgd_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
end = timer()
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')
print('Full time consumption [s]: ', round((end - start),3))
print('---------------------------------------------------------------------')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best estimator: SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.001,
             eta0=0.08, fit_intercept=True, l1_ratio=0.15,
             learning_rate='adaptive', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Best params: {'eta0': 0.08, 'learning_rate': 'adaptive'}
---------------------------------------------------------------------
17.68070516260644 {'eta0': 0.005, 'learning_rate': 'constant'}
17.82849647295309 {'eta0': 0.005, 'learning_rate': 'optimal'}
17.72116232294143 {'eta0': 0.005, 'learning_rate': 'invscaling'}
17.599063346591766 {'eta0': 0.005, 'learning_rate': 'adaptive'}
17.700342739153164 {'eta0': 0.01, 'learning_rate': 'constant'}
17.89549691830214 {'eta0': 0.01, 'learning_rate': 'optimal'}
17.67154452000128 {'eta0': 0.01, 'learning_rate': 'invscaling'}
17.5944287

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.001,
             eta0=0.08, fit_intercept=True, l1_ratio=0.15,
             learning_rate='adaptive', loss='epsilon_insensitive',
             max_iter=1000, n_iter_no_change=5, penalty='l2', power_t=0.25,
             random_state=None, shuffle=True, tol=0.001,
             validation_fraction=0.1, verbose=0, warm_start=False)
Train set: Regressor name, RMSE, R2
SGDRegressor 17.526 0.7
Test set: Regressor name, RMSE, R2
SGDRegressor 18.003 0.681
Time consumption [s]:  1.453
---------------------------------------------------------
---------------------------------------------------------------------
Full time consumption [s]:  95.776
---------------------------------------------------------------------


to conclude, we see four main points:

- low learning rates are not so important here (why?)
- PCA and collinear features remover are making the regression worse by quite a factor (best: train set: w.o.a. -> 70.0%; pca/cfr -> 64%; test set: w.o.a. -> 68.1%; pca/cfr -> 62%)
- with the sqrt-log feature adder, here the best results are gained (best: train set: w.o.a. -> 70.0%; w.o. -> 73.2%; test set: w.o.a. -> 68.0%; pca/cfr -> 71.0%)
- timing issues of course better with less features, but not too much difference. Full time consumption on Thinkpad Core i7 is ~95s